<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, TextVectorization
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [4]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [ ]:
items = list(filtered_data.items())


type_list = []
for row in items[:100]:
  second_key, second_value = row
  if second_value['type'] != None:
    print(second_value['type'])
    type_list.append(second_value['type'])

In [9]:
print(type_list[:150])

[['Creature'], ['Creature'], ['Artifact'], ['Instant'], ['Enchantment'], ['Instant'], ['Enchantment'], ['Creature'], ['Creature'], ['Creature'], ['Sorcery'], ['Instant'], ['Sorcery'], ['Land'], ['Land'], ['Artifact'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Enchantment'], ['Creature'], ['Creature'], ['Creature'], ['Instant'], ['Creature'], ['Creature'], ['Enchantment'], ['Instant'], ['Instant'], ['Creature'], ['Instant'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Enchantment'], ['Creature'], ['Instant'], ['Instant'], ['Land'], ['Instant'], ['Enchantment'], ['Enchantment'], ['Creature'], ['Creature'], ['Instant'], ['Instant'], ['Sorcery'], ['Land'], ['Instant'], ['Creature'], ['Instant'], ['Creature'], ['Sorcery'], ['Creature'], ['Creature'], ['Instant'], ['Enchantment'], ['Enchantment'], ['Sorcery'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], ['Creature'], 

In [21]:
from typing_extensions import Self
class TypeEncoderLayer(tf.keras.layers.Layer):
    def __init__(self,card_dict, **kwargs):
        super(TypeEncoderLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, output_mode='multi_hot')

        type_list = [self.clean_text(types) for card_name, card_data in card_dict.items() if card_data['type'] is not None for types in card_data['type']]


        self.vectorizer.adapt(type_list)
        self.embedding_layer = Embedding(input_dim=3000, output_dim=100)

    def clean_text(self, text):
        cleaned_text = tf.strings.regex_replace(text, '[^a-zA-Z0-9\s]', '')
        cleaned_text = tf.strings.lower(cleaned_text)
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return cleaned_text


# Example usage with a batch of texts
batch_texts = tf.constant(type_list)
cleaned_text_layer = TypeEncoderLayer(filtered_data)
cleaned_batch = cleaned_text_layer.call(batch_texts)
for i in range(len(cleaned_batch)):
  tf.print(cleaned_batch[i])

creature  zombie guest
creature  goblin guest
artifact  equipment
instant
enchantment
instant
enchantment
creature  bird rogue
legendary creature  insect druid
legendary creature  astartes warrior
sorcery
instant
sorcery
land
land
artifact
creature  zombie
creature  gargoyle
creature  griffin
creature  human cleric
creature  human monk
legendary creature  human warrior
enchantment  aura
creature  tyranid mutant
creature  human elf shaman
creature  human insect
instant
creature  eye
creature  demon
enchantment
instant
instant
creature  fish horror
instant
snow creature  treefolk
creature  horror
creature  horror
legendary creature  elf horror
creature  elemental
enchantment  aura
legendary creature  octopus noble
instant
instant
land  desert
instant
enchantment
enchantment
creature  thrull cleric
creature  lammasu
instant
instant
sorcery
legendary land
instant
creature  avatar
instant
creature  human
sorcery
legendary creature  spirit
creature  cat cleric
instant
enchantment
enchantment

In [ ]:
print(cleaned_text_layer.vectorizer.get_vocabulary())